## 1.2 读取ROOT的tree数据，进行逐事件分析
### 目的：
1. 学习进行逐事件分析的方法
2. 掌握中子探测器位置刻度、飞行时间刻度的方法
3. 学习数据分析的思路

``` cpp
//将下列代码保存到readTree.cc
//在ROOT环境下 .x readTree.cc

TH1D *hTOF;//如果将hTOF定义写在函数体内 hTOF->Draw()将不会显示
void readTree()
{
// 1.打开文件，得到TTree指针
  TFile *ipf=new TFile("tree.root");//打开ROOT文件
  TTree *tree=(TTree*)ipf->Get("tree");//得到名字为“tree”的TTree指针

//2. 声明tree的Branch变量

  Double_t x;
  Double_t e;
  int pid;
  Double_t tof, ctof;
  Double_t tu, td;
  Double_t qu, qd;
  
//3. 将变量指向对应Branch的地址
  tree->SetBranchAddress("ctof",&ctof);
  tree->SetBranchAddress("pid",&pid);
  tree->SetBranchAddress("tu",&tu);   
  tree->SetBranchAddress("td",&td);
  tree->SetBranchAddress("qu",&qu);   
  tree->SetBranchAddress("qd",&qd);

  //Histogram
  hTOF=new TH1D("hTOF","Time of flight", 1000,0,100);
  
  //将新数据写入新的ROOT文件 -对应的代码用 ////标出
  //// //calibration parameters
  //// Double_t a,b;
  //// ... ... ...
  //// //new tree parameters
  //// Double_t tx,qx,ce;
  //// ... ... ...    
  //// TFile *opf=new TFile("tree2.root","recreate");
  //// TTree *opt=new TTree("tree","tree");
  //// opt->SetBrach("tx",&tx,"tx/D");
  //// ... ... ...
    
    
  //4. 逐事件读取tree的branch数据
  Long64_t nentries=tree->GetEntries();//得到事件总数
  for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);//将第jentry个事件数据填入对应变量，每次变量值会变成当前事件对应的数据。
    hTOF->Fill(ctof);
    //// // calculate new parameters
    //// tx=tu-td;
    //// ... ... ...
      
    //// opt->Fill();//fill new parameter to TTree* opt
      
    if(jentry%100000==0) cout<<"process "<<jentry<<" of "<<nentries<<endl;
  }

  hTOF->Draw();
  ipt->Close();
  //// opt->Write();
  //// opf->Close();
}

 ```
 ### 练习：
  - 理解上述1.-4.步骤的逻辑关系。自行练习将上一步生成ROOT文件打开逐事件读出数据。
  - 理解如何在模拟中加入探测器分辨率信息。

In [1]:
%jsroot on
TH1D *tdiff=new TH1D("tdiff","td-tu",140,-20,50);  
TCanvas *c1=new TCanvas("c1","c1");

Double_t tu, td;
Double_t ctof,x;
int pid;
TFile *ipf=new TFile("tree.root");//打开ROOT文件
TTree *tree=(TTree*)ipf->Get("tree");//得到tree的指针

tree->SetBranchAddress("tu",&tu);   
tree->SetBranchAddress("td",&td);
tree->SetBranchAddress("ctof",&ctof);
tree->SetBranchAddress("x",&x);
tree->SetBranchAddress("pid",&pid);

In [2]:
Long64_t nentries=tree->GetEntries();//得到事件总数
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    tdiff->Fill(td-tu);  // if(ng==1) tx->Fill(tu-td), 只写入满足给定条件的事件      
  }
tdiff->Draw();
c1->Draw();

### 确定均匀分布边界的方法-微分法
- 原理见参考文献2.

In [3]:
TH1D *dtd=new TH1D("dtd","dt/dx",141,-20.25,50.25);
for(int i=1;i<tdiff->GetNbinsX();i++) {
    Double_t df=tdiff->GetBinContent(i+1)-tdiff->GetBinContent(i);
    dtd->Fill(tdiff->GetBinLowEdge(i+1),df);
}
dtd->Sumw2(0);
dtd->Draw();
dtd->Fit("gaus","","",-14,-11);//txl
c1->Draw();

 FCN=11.4095 FROM MIGRAD    STATUS=CONVERGED      67 CALLS          68 TOTAL
                     EDM=2.84899e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.00452e+02   1.30535e+01   1.75061e-02   1.52294e-05
   2  Mean        -1.17749e+01   2.38779e-02   3.85971e-05  -5.25171e-03
   3  Sigma        6.03011e-01   2.05075e-02   1.51564e-05   1.27386e-03


In [4]:
dtd->Fit("gaus","","",40,43);//为什么拟合有问题？
c1->Draw();

 FCN=1003 FROM HESSE     STATUS=FAILED         11 CALLS         131 TOTAL
                     EDM=0    STRATEGY= 1  ERROR MATRIX UNCERTAINTY 100.0 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant    -8.95914e+05   1.41421e+00   0.00000e+00   0.00000e+00
   2  Mean        -1.17749e+01   1.41421e+00   0.00000e+00   0.00000e+00
   3  Sigma        6.03011e-01   2.84295e+00   0.00000e+00   0.00000e+00


In [5]:
TF1 *f1 = new TF1("f1","[0]*TMath::Exp(-0.5*((x-[1])/[2])^2)",40,43);
//gaus:f(x) = p0*exp(-0.5*((x-p1)/p2)^2)

In [6]:
//进行参数拟合时，设置合理的初始参数至关重要！
c1->SetLogy(0);
f1->SetParameter(0,-350);
f1->SetParameter(1,41.5);
f1->SetParameter(2,0.5);
dtd->Fit("f1","R");
dtd->Draw();
c1->Draw();

 FCN=92.4636 FROM MIGRAD    STATUS=CONVERGED      69 CALLS          70 TOTAL
                     EDM=1.83854e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -3.03520e+02   1.18742e+01   4.74813e-02  -1.60223e-05
   2  p1           4.15780e+01   2.25280e-02   1.06110e-04  -4.11698e-03
   3  p2           6.15659e-01   1.54281e-02   6.18112e-05   5.88651e-03


### 探测器时间差-位置的刻度方法
- $t_x=t_d-t_u$
- $t_{xl}=-11.76,$ $\space t_{xr}=41.57$ 
- $t_{xoff}=(t_{xl}+t_{xr})/2=14.75$.

$$
x=\frac{2L}{t_{xr}-t_{xl}}*(t_x-t_{xoff})=3.750*(t_x-14.90)
$$

In [7]:
TH1D *htx=new TH1D("htx","htx",500,-120,120);
TH2D *hdx=new TH2D("hdx","htx-hx:hx",100,-20,20,500,-120,120);
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.750*(td-tu-14.90);
    htx->Fill(tx);
    hdx->Fill(tx-x,x);//difference
  }
htx->Draw();
c1->Draw();

### 评估计算结果
- 实际参数 x,计算参数tx
- 观察 tx-x 是否与x有关联性：二维图
- 观察 tx-x的中心值是否为零：一维图

In [8]:
hdx->Draw("colz");//为一条竖线，没有关联
c1->Draw();

In [9]:
TH1D *hdx1=hdx->ProjectionX("projx of hdx");
hdx1->Draw();
hdx1->Fit("gaus");//观察mean值是否接近零，如果偏离较大，需要检查上述拟合步骤是否合理。
c1->Draw();

 FCN=43.6928 FROM MIGRAD    STATUS=CONVERGED      58 CALLS          59 TOTAL
                     EDM=1.16699e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.09246e+03   2.74159e+01   7.32298e-02  -6.05229e-07
   2  Mean        -7.30447e-03   7.11690e-03   2.32317e-05   4.16698e-05
   3  Sigma        2.24909e+00   5.00189e-03   1.97903e-06  -2.23198e-02


### TOF绝对刻度的方法
实际探测器两端时间信号：
- 实际时间信号还具有一个额外附加的常数项，$T_{Off}$：来源包括PMT度越时间，和电缆，电子学的传输时间。
 - 这个常数项一般是无法直接测定的，需要由实验数据来计算得到。
- $t_u=TOF+(L-x)/v_{sc}+T_{uoff}$
- $t_d=TOF+(L+x)/v_{sc}+T_{doff}$
- $TOF=(t_u+t_d)/2 -L/v_{sc}-(T_{uoff}+T_{doff})/2$
- 将上式常数项设为C，$TOF=(t_u+t_d)/2 +C$
#### 光子打到探测器中心时，$TOF_0=d_0*c$, $d_0$靶点到探测器中心距离。 由此可以确定C
- 中心平均飞行距离502.5cm, $TOF_0=15.075 ns$

In [10]:
TH2D *hgtofx=new TH2D("hgtofx","hgtofx",100,-120,120,100,39,45);
TH1D *hgctof=new TH1D("hgctof","hgctof",100,39,45);

In [11]:
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.745*(td-tu-14.75);
    if(ctof>40&& ctof<43) { 
        hgtofx->Fill(tx,ctof);
        if(abs(tx)<5) hgctof->Fill(ctof);//gamma hits the center of the det.
    }
  }
hgtofx->Draw("colz");
c1->Draw();

#### 目标：确定C，并进行飞行距离修正(消除tof与x的关联性，曲线变平直)
- 确定探测器中心位置的gamma峰位TOF(0)

In [12]:
hgctof->Draw();
hgctof->Fit("gaus");
c1->Draw();

 FCN=31.1276 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                     EDM=1.59749e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.21272e+02   3.77161e+00   8.62819e-03  -1.19746e-04
   2  Mean         4.12759e+01   7.76066e-03   2.14533e-05   3.68030e-02
   3  Sigma        2.99076e-01   5.31993e-03   1.34009e-05  -9.50346e-02


- $TOF(0)=41.278$ ns
- $C=TOF_0-TOF(0)=-26.203$ ns

In [13]:
TH2D *hgtofcx=new TH2D("hgtofcx","corrected TOF",100,-120,120,100,12,18);
TH1D *htofc=new TH1D("htofc","htof",200,12,100);

In [14]:
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.745*(td-tu-14.75);
    Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);
    Double_t tofc=(ctof-26.203)/d*500.;//normalized to 500cm
    hgtofcx->Fill(tx,tofc);//gamma hits the center of the det.
    htofc->Fill(tofc);
  }
hgtofcx->Draw("colz");//tofc与x之间无关联
c1->Draw();

In [15]:
c1->SetLogy();
htofc->Draw();//修正后的飞行时间谱。
c1->Draw();